# **Mitigation Bias using Fairlet clustering**


## **Load the data**

In [1]:
# Imports
import numpy as np
import pandas as pd

#sys
import sys
sys.path.append('../../')

We will start by importing the adult dataset, which we host on our library. The adult dataset contains a set of informations extract from US 1994 Census database. It includes personal information about the individuals, specifically sex, race, and education. In this tutorial we will perform unsupervised learning to cluster the data, then measure whether this clustering contains gender or race information (clustering bias).

In [2]:
# Get data
from holisticai.datasets import load_adult
df = load_adult()['frame']
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


## **Preprocess data and Train a model**

In [3]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
# Dataset
dataset = load_adult()

# Dataframe
df = pd.concat([dataset["data"], dataset["target"]], axis=1)
protected_variables = ["sex", "race"]
output_variable = ["class"]

# Simple preprocessing
y = df[output_variable].replace({">50K": 1, "<=50K": 0})
X = pd.get_dummies(df.drop(protected_variables + output_variable, axis=1))
group = ["sex"]
group_a = df[group] == "Female"
group_b = df[group] == "Male"
data = [X, y, group_a, group_b]

# Train test split
dataset = train_test_split(*data, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]

In [5]:

from sklearn.cluster import KMeans
from holisticai.bias.metrics import clustering_bias_metrics

X_train, _, group_a_train, group_b_train = train_data
# we choose to use 4 clusters
model = KMeans(n_clusters = 4)
model.fit(X_train)

# test data
X, _, group_a, group_b = test_data

# predict
y_pred = model.predict(X)

centroids = model.cluster_centers_

# Computing Bias Metrics for Train Data
dfm = clustering_bias_metrics(group_a_train, 
                              group_b_train, 
                              model.labels_, 
                              data = X_train, 
                              centroids = centroids, 
                              metric_type = 'equal_outcome')

# Computing Bias Metrics for Test Data
dfm_ts = clustering_bias_metrics(group_a, 
                                 group_b, 
                                 y_pred, 
                                 data = X, 
                                 centroids = centroids, 
                                 metric_type = 'equal_outcome')

# Using Fairlet Clustering Preprocessing

In [6]:
from holisticai.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from holisticai.bias.mitigation import FairletClusteringPreprocessing
from holisticai.bias.mitigation.commons.fairlet_clustering.clustering._kmedoids import KMedoids
from sklearn.cluster import KMeans
decomposition = FairletClusteringPreprocessing(decomposition='Scalable', p=10, q=21, 
                                          seed=42)
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_preprocessing', decomposition),
    ('cluster', KMeans(n_clusters=4))])

pipeline.fit(X_train, bm__group_a = group_a_train, bm__group_b = group_b_train)

c:\Users\Cristian\anaconda3\envs\metrics\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline(steps=[('scaler', StandardScaler()),
                ('bm_preprocessing',
                 FairletClusteringPreprocessing(decomposition=<holisticai.bias.mitigation.commons.fairlet_clustering.decomposition._scalable.ScalableFairletDecomposition object at 0x00000250E9AEA460>,
                                                p=10, q=21, seed=42)),
                ('cluster',
                 <holisticai.pipeline.handlers._estimator.WEstimator object at 0x0000025098D7F100>)])

In [11]:
centroids2 = pipeline['cluster'].cluster_centers_
y_pred2 = pipeline.predict(X)
dfm2 = clustering_bias_metrics(group_a_train, 
                               group_b_train, 
                               pipeline['cluster'].labels_, 
                               data = X_train, 
                               centroids = centroids2, 
                               metric_type = 'both')

dfm2_ts = clustering_bias_metrics(group_a, 
                                  group_b, 
                                  y_pred2, 
                                  data = X, 
                                  centroids = centroids2, 
                                  metric_type = 'both')

In [12]:
from holisticai.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from holisticai.bias.mitigation import FairletClustering

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('bm_inprocessing', FairletClustering(decomposition='Scalable', clustering_model='KMedoids', 
                                          p=10, q=21, n_clusters=4, seed=50))])
pipeline.fit(X_train, bm__group_a = group_a_train, bm__group_b = group_b_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('bm_inprocessing',
                 <holisticai.pipeline.handlers._estimator.WEstimator object at 0x00000250999CD610>)])

In [13]:
centroids3 = pipeline['bm_inprocessing'].cluster_centers_
y_pred3 = pipeline.predict(X)
dfm3 = clustering_bias_metrics(group_a_train, 
                               group_b_train, 
                               pipeline['bm_inprocessing'].labels_, 
                               data = X_train, 
                               centroids = centroids3, 
                               metric_type = 'both')

dfm3_ts = clustering_bias_metrics(group_a, 
                                  group_b, 
                                  y_pred3, 
                                  data = X, 
                                  centroids = centroids3, 
                                  metric_type = 'both')

In [14]:
metrics = pd.concat([dfm["Value"], dfm_ts["Value"], 
                     dfm2["Value"], dfm2_ts["Value"], 
                     dfm3["Value"], dfm3_ts],axis=1)
metrics.columns= ['Baseline (Train)','Baseline (Test)', 
                  'Fairlet-Pre (Train)', 'Fairlet-Pre (Test)', 
                  'Fairlet-In (Train)', 'Fairlet-In (Test)', 
                  'Reference']
metrics

,Baseline (Train),Baseline (Test),Fairlet-Pre (Train),Fairlet-Pre (Test),Fairlet-In (Train),Fairlet-In (Test),Reference
Metric,,,,,,,
Cluster Balance,0.806236,0.803901,0.963644,0.757522,0.971946,0.336676,1
Minimum Cluster Ratio,0.365342,0.361111,0.476671,0.333333,0.476190,0.125000,1
Cluster Distribution Total Variation,0.030031,0.009620,0.022001,0.150223,0.000133,0.001213,0
Cluster Distribution KL Div,0.003120,0.001365,0.001034,0.048114,0.000003,0.000595,0
Social Fairness Ratio,0.968921,0.955704,0.962764,0.983281,0.962764,0.983281,1
Silhouette Difference,0.003002,0.004246,-0.001225,-0.001594,-0.004488,-0.006177,0
